In [78]:
import os
import pandas as pd
from datetime import datetime
from sklearn.impute import KNNImputer
import random
import numpy as np

In [1]:
"""

Data Source and variable descriptions at https://www.ers.usda.gov/data-products/county-level-data-sets/download-data/

"""
mob_d=pd.read_csv("https://raw.githubusercontent.com/dewaranch/Covid/master/old-format/raw_data/Global_Mobility_Report_05_01_2020.csv")
FIPS_d=pd.read_csv("C:/Users/sh777/Documents/GitHub/Covid/raw_data/David/FIPS.csv")
Edu_d=pd.read_csv("C:/Users/sh777/Documents/GitHub/Covid/raw_data/David/Education.csv")
City_d=pd.read_csv("C:/Users/sh777/Documents/GitHub/Covid/raw_data/David/City.csv")
Pop_d=pd.read_csv("C:/Users/sh777/Documents/GitHub/Covid/raw_data/David/Population.csv")
Pov_d=pd.read_csv("C:/Users/sh777/Documents/GitHub/Covid/raw_data/David/Poverty.csv")
    
mob_d=mob_d[(mob_d["country_region_code"]=="US") & (mob_d["sub_region_1"]) & (mob_d["sub_region_2"])]
FIPS_d["statey"]=FIPS_d.iloc[:,3]+FIPS_d.iloc[:,4]
mob_d["statey"]=mob_d["sub_region_1"]+mob_d["sub_region_2"]
merg_d=mob_d.merge(FIPS_d,on="statey")
merg_d=merg_d[['County Subdivision Code (FIPS)','sub_region_1', 'sub_region_2','date', 'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline','parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline','workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline']]
merg_d.columns=['FIPS', 'State', 'County','Date', 'Mobility_Retail_and_Recreation',
       'Mobility_Grocery_and_Pharmacy','Mobility_Parks',
       'Mobility_Transit_Stations','Mobility_Workplaces','Mobility_Residential']

Pov_d=Pov_d[['FIPStxt', 'PCTPOVALL_2018', 'PCTPOV017_2018', 'MEDHHINC_2018']]
Pov_d.columns=['FIPS','Poverty_Percent','Poverty_Percent_Age0_to_Age17','Poverty_Median_Household_Income']

Edu_d=Edu_d[['FIPS', 
       'Percent of adults with less than a high school diploma, 2014-18',
       'Percent of adults with a high school diploma only, 2014-18',
       "Percent of adults completing some college or associate's degree, 2014-18",
       "Percent of adults with a bachelor's degree or higher, 2014-18"]]
Edu_d.columns=['FIPS',"Edu_Percent_Less_HS","Edu_Percent_HS","Edu_Percent_AD","Edu_Percent_BD_or_Higher"]

Pop_d=Pop_d[['FIPS', 'POP_ESTIMATE_2018', 'N_POP_CHG_2018', 'Births_2018', 'Deaths_2018',
             'INTERNATIONAL_MIG_2018', 'DOMESTIC_MIG_2018']]
Pop_d.columns=['FIPS', 'Pop', 'Pop_Change', 'Pop_Birth', 'Pop_Death',
             'Pop_Int_Mig_Net', 'Pop_Dom_Mig_Net']

City_d=City_d[["FIPS code","2013 code"]]
City_d.columns=["FIPS","City_Code"]

total_d=merg_d.merge(Pov_d,on="FIPS").merge(Edu_d,on="FIPS").merge(Pop_d,on="FIPS").merge(City_d,on="FIPS")

os.chdir("C:/Users/sh777/Documents/OneDrive/Harvard/SP20/BST 263")

COVID_d=pd.read_csv("https://raw.githubusercontent.com/dewaranch/Covid/master/raw_data/covid_data.csv")
COVID_d=COVID_d[['fips', 'lat', 'long_', 'confirmed', 'deaths','date']]
COVID_d.columns=['FIPS', 'Latitude', 'Longitude', 'COVID_Confirm', 'COVID_Death','Date']
real_total_d=pd.merge(total_d,COVID_d,on=['FIPS','Date'])

date_object = [(datetime.combine(datetime.strptime(i, '%Y-%m-%d').date(),datetime.min.time())-datetime.fromordinal(737456)).days for i in real_total_d["Date"]]
real_total_d["Date_Integer"]=date_object
real_total_d.sort_values(["FIPS",'Date_Integer'],ascending=[True,True])
n_row=len(real_total_d.FIPS)
real_total_d["COVID_newConfirm"]=n_row*[0]
for i in range(n_row):
    if i==0: pass
    else:
        if real_total_d["FIPS"][i]==real_total_d["FIPS"][i-1]:
            real_total_d["COVID_newConfirm"][i]=abs(real_total_d["COVID_Confirm"][i]-real_total_d["COVID_Confirm"][i-1])
        else:
            real_total_d["COVID_newConfirm"][i]=real_total_d["COVID_Confirm"][i]
            
real_total_d=real_total_d[real_total_d.COVID_Death!=0].reset_index()
n_row2=len(real_total_d.FIPS)
real_total_d["Death_Index"]=n_row2*[0]
for i in range(n_row2):
    if i==0: pass
    else:
        if real_total_d["FIPS"][i]==real_total_d["FIPS"][i-1]:
            real_total_d["Death_Index"][i]=real_total_d["Death_Index"][i-1]+1
        else: pass
real_total_d=real_total_d[real_total_d.Death_Index<4].reset_index()

test_left_d=real_total_d[['FIPS', 'State', 'County', 'Poverty_Percent',
       'Poverty_Percent_Age0_to_Age17', 'Poverty_Median_Household_Income',
       'Edu_Percent_Less_HS', 'Edu_Percent_HS', 'Edu_Percent_AD',
       'Edu_Percent_BD_or_Higher', 'Pop', 'Pop_Change', 'Pop_Birth',
       'Pop_Death', 'Pop_Int_Mig_Net', 'Pop_Dom_Mig_Net', 'City_Code',
       'Latitude', 'Longitude']].drop_duplicates()

test_right_d=real_total_d.pivot(index='FIPS',columns='Death_Index',values=['Mobility_Retail_and_Recreation','Mobility_Grocery_and_Pharmacy',
       'Mobility_Parks', 'Mobility_Transit_Stations', 'Mobility_Workplaces',
       'Mobility_Residential','COVID_Confirm', 'COVID_Death','COVID_newConfirm'])

test_d_merged=test_left_d.merge(test_right_d,on="FIPS")
test_d_merged.to_csv("Merged_Wide2.csv",index=False)

C:\Users\sh777\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


HTTPError: HTTP Error 404: Not Found

In [121]:
column_list=['lat', 'lon', 'POP_LATITUDE', 'POP_LONGITUDE',
             'PopulationEstimate2018','PopTotalMale2017', 'PopTotalFemale2017', 'FracMale2017','PopulationEstimate65+2017',
             'PopulationDensityperSqMile2010','CensusPopulation2010', 'MedianAge2010', '#EligibleforMedicare2018',
             'MedicareEnrollment,AgedTot2017', '3-YrDiabetes2015-17','DiabetesPercentage', 'HeartDiseaseMortality',
             'StrokeMortality','Smokers_Percentage', 'RespMortalityRate2014', '#FTEHospitalTotal2017',
             "TotalM.D.'s,TotNon-FedandFed2017", '#HospParticipatinginNetwork2017','#Hospitals', '#ICU_beds',
             'dem_to_rep_ratio', 'PopMale<52010','PopFmle<52010', 'PopMale5-92010', 'PopFmle5-92010', 'PopMale10-142010',
             'PopFmle10-142010', 'PopMale15-192010', 'PopFmle15-192010','PopMale20-242010', 'PopFmle20-242010', 
             'PopMale25-292010','PopFmle25-292010', 'PopMale30-342010', 'PopFmle30-342010','PopMale35-442010', 
             'PopFmle35-442010', 'PopMale45-542010','PopFmle45-542010', 'PopMale55-592010', 'PopFmle55-592010',
             'PopMale60-642010', 'PopFmle60-642010', 'PopMale65-742010','PopFmle65-742010', 'PopMale75-842010',
             'PopFmle75-842010', 'PopMale>842010', 'PopFmle>842010', '3-YrMortalityAge<1Year2015-17', 
             '3-YrMortalityAge1-4Years2015-17', '3-YrMortalityAge5-14Years2015-17', '3-YrMortalityAge15-24Years2015-17',
             '3-YrMortalityAge25-34Years2015-17','3-YrMortalityAge35-44Years2015-17','3-YrMortalityAge45-54Years2015-17',
             '3-YrMortalityAge55-64Years2015-17','3-YrMortalityAge65-74Years2015-17','3-YrMortalityAge75-84Years2015-17',
             '3-YrMortalityAge85+Years2015-17','mortality2015-17Estimated', 'SVIPercentile', 'HPSAShortage','HPSAServedPop',
             'HPSAUnderservedPop']

column_list2=['countyFIPS', 'lat', 'lon', 'POP_LATITUDE','POP_LONGITUDE', 'Rural-UrbanContinuumCode2013', 'PopulationEstimate2018',
              'PopTotalMale2017', 'PopTotalFemale2017', 'FracMale2017','PopulationEstimate65+2017',
              'PopulationDensityperSqMile2010','CensusPopulation2010', 'MedianAge2010', '#EligibleforMedicare2018',
              'MedicareEnrollment,AgedTot2017', '3-YrDiabetes2015-17','DiabetesPercentage', 'HeartDiseaseMortality',
              'StrokeMortality','Smokers_Percentage', 'RespMortalityRate2014', '#FTEHospitalTotal2017',
              "TotalM.D.'s,TotNon-FedandFed2017", '#HospParticipatinginNetwork2017',
              '#Hospitals', '#ICU_beds', 'dem_to_rep_ratio', 'PopMale<52010','PopFmle<52010', 'PopMale5-92010',
              'PopFmle5-92010', 'PopMale10-142010','PopFmle10-142010', 'PopMale15-192010', 'PopFmle15-192010',
              'PopMale20-242010', 'PopFmle20-242010', 'PopMale25-292010','PopFmle25-292010', 'PopMale30-342010',
              'PopFmle30-342010','PopMale35-442010', 'PopFmle35-442010', 'PopMale45-542010','PopFmle45-542010',
              'PopMale55-592010', 'PopFmle55-592010','PopMale60-642010', 'PopFmle60-642010', 'PopMale65-742010',
              'PopFmle65-742010', 'PopMale75-842010', 'PopFmle75-842010','PopMale>842010', 'PopFmle>842010',
              '3-YrMortalityAge<1Year2015-17','3-YrMortalityAge1-4Years2015-17', '3-YrMortalityAge5-14Years2015-17',
              '3-YrMortalityAge15-24Years2015-17','3-YrMortalityAge25-34Years2015-17','3-YrMortalityAge35-44Years2015-17',
              '3-YrMortalityAge45-54Years2015-17','3-YrMortalityAge55-64Years2015-17','3-YrMortalityAge65-74Years2015-17',
              '3-YrMortalityAge75-84Years2015-17', '3-YrMortalityAge85+Years2015-17',
              'mortality2015-17Estimated', 'SVIPercentile', 'HPSAShortage','HPSAServedPop', 'HPSAUnderservedPop']

In [123]:
station_d=pd.read_csv("https://raw.githubusercontent.com/dewaranch/Covid/master/data/processed/abridged_stationary_features.tsv",
                      sep="\t")
temp_d=station_d[column_list2]
imputer = KNNImputer(n_neighbors=2, weights="uniform")
temp_d_filled=imputer.fit_transform(temp_d)
temp_d_filled=pd.DataFrame(data=temp_d_filled,columns=column_list2)
for i,k in enumerate(temp_d_filled.isnull().sum(axis=0)):
    if k!=0:
        print(station_d.columns[i],k)
n_row=len(temp_d_filled)
for i in range(500):
    temp_copy[random.choice(column_list)][random.randint(0,n_row-1)]=np.nan
    
imputer = KNNImputer(n_neighbors=2, weights="uniform")
temp_copy_filled=imputer.fit_transform(temp_copy)

temp_copy_filled=pd.DataFrame(data=temp_copy_filled,columns=column_list2)
hi=((temp_copy_filled-temp_copy_filled.mean())/temp_copy_filled.std()-(temp_d_filled-temp_d_filled.mean())/temp_d_filled.std())**2
hi.values.sum()

In [135]:
temp_d[~temp_d.isnull().any(axis=1)]

,countyFIPS,lat,lon,POP_LATITUDE,POP_LONGITUDE,Rural-UrbanContinuumCode2013,PopulationEstimate2018,PopTotalMale2017,PopTotalFemale2017,FracMale2017,...,3-YrMortalityAge45-54Years2015-17,3-YrMortalityAge55-64Years2015-17,3-YrMortalityAge65-74Years2015-17,3-YrMortalityAge75-84Years2015-17,3-YrMortalityAge85+Years2015-17,mortality2015-17Estimated,SVIPercentile,HPSAShortage,HPSAServedPop,HPSAUnderservedPop
966,20173,37.665350,-97.469220,37.678808,-97.344603,2.0,513607.0,254121.0,259566.0,0.494700,...,290.0,666.0,821.0,986.0,1339.0,268.783456,0.5357,33.5573,73028.0,100673.0
2194,40143,36.133837,-95.942805,36.105918,-95.914798,2.0,648360.0,315370.0,330896.0,0.487988,...,406.0,862.0,1065.0,1320.0,1673.0,370.036215,0.6159,4.8300,204030.0,14484.0
2438,47037,36.175251,-86.778785,36.140596,-86.745450,1.0,692587.0,333380.0,357863.0,0.482291,...,443.0,879.0,979.0,1117.0,1430.0,344.431089,0.5363,18.9500,186180.0,56854.0


In [147]:
station_d.shape

(3142, 75)

In [146]:
for i,j in enumerate(station_d.isnull().sum(axis=0)):
    print(station_d.columns[i],j)

countyFIPS 0
StateName 0
lat 67
lon 67
POP_LATITUDE 0
POP_LONGITUDE 0
CensusRegionName 1
CensusDivisionName 1
Rural-UrbanContinuumCode2013 1
PopulationEstimate2018 1
PopTotalMale2017 1
PopTotalFemale2017 1
FracMale2017 1
PopulationEstimate65+2017 1
PopulationDensityperSqMile2010 1
CensusPopulation2010 1
MedianAge2010 1
#EligibleforMedicare2018 8
MedicareEnrollment,AgedTot2017 22
3-YrDiabetes2015-17 1721
DiabetesPercentage 2
HeartDiseaseMortality 7
StrokeMortality 8
Smokers_Percentage 1
RespMortalityRate2014 1
#FTEHospitalTotal2017 1
TotalM.D.'s,TotNon-FedandFed2017 1
#HospParticipatinginNetwork2017 1
#Hospitals 1
#ICU_beds 1
dem_to_rep_ratio 27
PopMale<52010 1
PopFmle<52010 1
PopMale5-92010 1
PopFmle5-92010 1
PopMale10-142010 1
PopFmle10-142010 1
PopMale15-192010 1
PopFmle15-192010 1
PopMale20-242010 1
PopFmle20-242010 1
PopMale25-292010 1
PopFmle25-292010 1
PopMale30-342010 1
PopFmle30-342010 1
PopMale35-442010 1
PopFmle35-442010 1
PopMale45-542010 1
PopFmle45-542010 1
PopMale55-59201

In [132]:
~(station_d.isnull().any(axis=1))

0       False
1       False
2       False
3       False
4       False
        ...  
3137    False
3138    False
3139    False
3140    False
3141    False
Length: 3142, dtype: bool